In [33]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 2 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:16.04

MAINTAINER Amazon AI <sage-learner@amazon.com>


RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# Here we get all python packages.
# There's substantial overlap between scipy and numpy that we eliminate by
# linking them together. Likewise, pip leaves the install caches populated which uses
# a significant amount of space. These optimizations save a fair amount of space in the
# image, which reduces start up time.
RUN wget https://bootstrap.pypa.io/get-pip.py && python3 get-pip.py && \
    pip3 install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.3 pandas flask gevent gunicorn && \
        (cd /usr/local/lib/python3.5/dist-packages/scipy/.libs; rm *; ln ../

### Building and registering the container

The following shell code shows how to build the container image using `docker build` and push the container image to ECR using `docker push`. This code is also available as the shell script `container/build-and-push.sh`, which you can run as `build-and-push.sh decision_trees_sample` to build the image `decision_trees_sample`. 

This code looks for an ECR repository in the account you're using and the current default region (if you're using a SageMaker notebook instance, this will be the region where the notebook instance was created). If the repository doesn't exist, the script will create it.

In [51]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-decision-trees

cd container

chmod +x decision_trees/train
chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded

Step 1/9 : FROM ubuntu:16.04
 ---> 005d2078bdfa
Step 2/9 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> 26d4bb9f720c
Step 3/9 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 9436d212427c
Step 4/9 : RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py &&     pip install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gevent gunicorn &&         (cd /usr/local/lib/python2.7/dist-packages/scipy/.libs; rm *; ln ../../numpy/.libs/* .) &&         rm -rf /root/.cache
 ---> Using cache
 ---> 4d2c0e030856
Step 5/9 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 56c64de4487d
Step 6/9 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> e5d509e6ab5a
Step 7/9 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> 43a2ada47ae0
Step 8/9 : COPY decision_trees /opt/program

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [7]:
# S3 prefix
prefix = 'DEMO-scikit-byo-iris'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Create the session

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [8]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [18]:
import boto3 
sagemaker_low_level = boto3.client('sagemaker')

image_uri = "667283206402.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-decision-trees"
modelName = "HostingTrainingExampleModel"

model = sagemaker_low_level.create_model(
            ModelName=modelName,
            ExecutionRoleArn=role, 
            PrimaryContainer={
                    'ContainerHostname': 'HostingTrainingExampleModel',
                    'Image': image_uri,
                    'Mode': 'SingleModel',
                    'ModelDataUrl': 's3://sagemaker-training-01/HostingTrainingModel/model.tar.gz',
                }    
            )

ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Cannot create already existing model "arn:aws:sagemaker:eu-west-1:667283206402:model/hostingtrainingexamplemodel".

In [29]:
endpointConfigName="HostingTrainingExampleConfig1"
endpoint_config = sagemaker_low_level.create_endpoint_config(
                        EndpointConfigName=endpointConfigName,
                        ProductionVariants=[{
                            'InstanceType':'ml.p2.xlarge',
                            'InitialInstanceCount':1,
                            'ModelName':modelName,
                            'VariantName':'AllTraffic'}]
                    )

In [52]:
%%time
import time

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_name = 'HostingTrainingExampleEndpoint-' + timestamp
print('Endpoint name: {}'.format(endpoint_name))

endpoint_params = {
    'EndpointName': endpoint_name,
    'EndpointConfigName': endpointConfigName,
}

endpoint_response = sagemaker.create_endpoint(**endpoint_params)
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))

Endpoint name: HostingTrainingExampleEndpoint--2020-06-07-13-22-18
EndpointArn = arn:aws:sagemaker:eu-west-1:667283206402:endpoint/hostingtrainingexampleendpoint--2020-06-07-13-22-18
CPU times: user 15.5 ms, sys: 0 ns, total: 15.5 ms
Wall time: 220 ms


In [53]:
# get the status of the endpoint
response = sagemaker.describe_endpoint(EndpointName=endpoint_name)
status = response['EndpointStatus']
print('EndpointStatus = {}'.format(status))
    
try:
    sagemaker.get_waiter('endpoint_in_service').wait(EndpointName=endpoint_name)
finally:
    resp = sagemaker.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Arn: " + resp['EndpointArn'])
    print("Create endpoint ended with status: " + status)
    
    if status != 'InService':
        message = sagemaker.describe_endpoint(EndpointName=endpoint_name)['FailureReason']
        print('Training failed with the following error: {}'.format(message))
        raise Exception('Endpoint creation did not succeed')

EndpointStatus = Creating
Arn: arn:aws:sagemaker:eu-west-1:667283206402:endpoint/hostingtrainingexampleendpoint--2020-06-07-13-22-18
Create endpoint ended with status: InService


In [54]:
import boto3
runtime = boto3.Session().client(service_name='runtime.sagemaker')

In [73]:
test_csv = '3.4,1.4,0.3,0'
response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/csv', 
                                   Body=test_csv)
result = response['Body'].read()

In [74]:
print('Predicted Sepal Length (cm):', result)
print('Actual Sepal Length (cm):', 4.6)

Predicted Sepal Length (cm): b'4.88749921150266\n'
Actual Sepal Length (cm): 4.6
